In [6]:
import pandas as pd

import osmosys.osmosys
import activityinfo
from activityinfo import Client
from activityinfo import FormProcessing
import importlib
import model.modelAI
from activityinfo.id import generate_id
import json
from requests.exceptions import HTTPError
import os.path

importlib.reload(osmosys.osmosys)
importlib.reload(activityinfo.FormProcessing)
importlib.reload(model.modelAI)

<module 'model.modelAI' from 'D:\\Proyectos\\OSMOSYS_2022\\IntegraciónAI\\AI_OsmosysIntegrationScripts\\model\\modelAI.py'>

In [7]:
## parameters
indicatorCodeAI = 'WA1_01'
month = 'JUNIO'
month_number = '2022-06'
year = 2022
test = True
print(
    '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')


--------------------------------------WA1_01------------------------------------------------------


In [8]:
# busco matchs
subformDf = osmosys.osmosys.getMatchSubforms(indicatorCodeAI)
## print(subformDf)
indicatorIdsOsmosys = subformDf.osmosys_indicador_id.unique()
subformSeries = subformDf.iloc[0]
formIdAI = subformSeries.form_id
parentFormIdAI = subformSeries.parent_form_id
indicatorIdAI = subformSeries.indicator_id
print("formIdAI: " + formIdAI)
print("parentFormIdAI: " + parentFormIdAI)
print("indicatorIdAI: " + indicatorIdAI)
print("indicatorIdsOsmosys: " + str(indicatorIdsOsmosys))

formIdAI: crevidcl4q3ommxdh
parentFormIdAI: culktyl4q3ommxdr
indicatorIdAI: ch42l00kylzqu5260
indicatorIdsOsmosys: [72 73 71]


In [9]:
## consulto los datos reportados
reportedDf = osmosys.osmosys.getIesPartnerCantonsByIndicatorsIdsOsmosysAndMonth(indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                                month=month, year=year)
print('reported data: ' + str(reportedDf.shape[0]))


reported data: 0


In [10]:
## obtengo los parents de AI
parentFormsIds = FormProcessing.getParentsIds(parentFormIdAI=parentFormIdAI, reportedDf=reportedDf)

In [11]:
## 2 fuentes por separado indicador 35
## contruyo las estructuras de datos
changesList = []
newIds = []
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    poblacion_meta = ["Refugiados/as y migrantes", "Comunidad de acogida"]
    modalidad_impl = 'cb53mqwkykwwihkk8',  ## kits

    WA1_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    WA1_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    WA1_01_RM_MM = int(df.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    WA1_01_RM_HH = int(df.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    WA1_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    WA1_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    WA1_01_CA_MM = int(dfCa.loc[df['age_gender'] == 'ADULTAS'].iloc[0].value_a)
    WA1_01_CA_HH = int(dfCa.loc[df['age_gender'] == 'ADULTOS'].iloc[0].value_a)

    commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
        0].value_a

    subform = model.modelAI.SubFormWA1_01(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        wash_individual='No',
        poblacion_meta=poblacion_meta,
        modalidad_impl=modalidad_impl,
        WA1_01_RM_NA=WA1_01_RM_NA,
        WA1_01_RM_NN=WA1_01_RM_NN,
        WA1_01_RM_MM=WA1_01_RM_MM,
        WA1_01_RM_HH=WA1_01_RM_HH,
        WA1_01_CA_NA=WA1_01_CA_NA,
        WA1_01_CA_NN=WA1_01_CA_NN,
        WA1_01_CA_MM=WA1_01_CA_MM,
        WA1_01_CA_HH=WA1_01_CA_HH,
        WA1_01_CUAL=commentary
    )
    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 0


In [12]:
## contruyo las estructuras de datos
for index, row in parentFormsIds.iterrows():
    # print(row)
    orgAcron = row.acronym
    cantonCode = row.canton_code
    df = osmosys.osmosys.getRefValues(year=year, month=month, orgOsmosys=orgAcron,
                                      indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)

    poblacion_meta = ['Refugiados/as y migrantes', 'Comunidad de acogida']

    PRPI1_01_RM_NA = int(df.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_RM_NN = int(df.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_RM_OTR = int(df.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)
    dfDiversidad = osmosys.osmosys.getRefLgbtiDiscapacitadosValues(year=year, month=month, orgOsmosys=orgAcron,
                                                                   indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                                   cantonCode=cantonCode)
    PRPI1_01_RM_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)
    PRPI1_01_RM_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'VENEZUELA')].iloc[0].value_a)

    dfCa = osmosys.osmosys.getCAValues(year=year, month=month, orgOsmosys=orgAcron,
                                       indicatorsIdsOmosys=indicatorIdsOsmosys, cantonCode=cantonCode)
    PRPI1_01_CA_NA = int(dfCa.loc[df['age_gender'] == 'NINAS'].iloc[0].value_a)
    PRPI1_01_CA_NN = int(dfCa.loc[df['age_gender'] == 'NINOS'].iloc[0].value_a)
    PRPI1_01_CA_OTR = int(dfCa.loc[df['age_gender'] == 'OTRO'].iloc[0].value_a)

    PRPI1_01_CA_LGBT = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'LGBTI') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CA_DS = int(dfDiversidad.loc[(dfDiversidad['diversity_type'] == 'DISCAPACITADOS') & (
            dfDiversidad['country_of_origin'] == 'ECUADOR')].iloc[0].value_a)
    PRPI1_01_CUAL = commentary = osmosys.osmosys.getCommentary(year=year, month=month, orgOsmosys=orgAcron,
                                                               indicatorsIdsOmosys=indicatorIdsOsmosys).iloc[
        0].value_a
    usd_transfer = int(osmosys.osmosys.getCBIBudget(year=year, month=month, orgOsmosys=orgAcron,
                                                    indicatorsIdsOmosys=indicatorIdsOsmosys,
                                                    cantonCode=cantonCode).iloc[0].budget)
    subform = model.modelAI.SubFormPRPI1_01_85(
        mes=month_number,
        colltmgkykvhxgij6=indicatorIdAI,
        rmrp='Si',
        covid='No',
        espacion_apoyo='No',
        poblacion_meta=poblacion_meta,
        modalidad_impl='cmo12q1kykwwosjka',  ## servicio
        mecanismos='c6gspwxkym0dbinm',
        transferencia='cib4u00kym0ixwew',
        usd_transfer=usd_transfer,
        PRPI1_01_RM_NA=PRPI1_01_RM_NA,
        PRPI1_01_RM_NN=PRPI1_01_RM_NN,
        PRPI1_01_RM_OTR=PRPI1_01_RM_OTR,
        PRPI1_01_RM_LGBT=PRPI1_01_RM_LGBT,
        PRPI1_01_RM_DS=PRPI1_01_RM_DS,
        PRPI1_01_CA_NA=PRPI1_01_CA_NA,
        PRPI1_01_CA_NN=PRPI1_01_CA_NN,
        PRPI1_01_CA_OTR=PRPI1_01_CA_OTR,
        PRPI1_01_CA_LGBT=PRPI1_01_CA_LGBT,
        PRPI1_01_CA_DS=PRPI1_01_CA_DS,
        PRPI1_01_CUAL=PRPI1_01_CUAL
    )

    newId = generate_id()
    newIds.append(newId)
    record = model.modelAI.Record(formId=formIdAI, recordId=newId, parentRecordId=row['@id'], fields=subform)
    changesList.append(record)
print('changes to charge: ' + str(len(changesList)))

changes to charge: 0


In [13]:
## creo respaldo
changes = model.modelAI.Changes(changesList)
finalJson = json.dumps(changes, default=model.modelAI.default)

In [14]:
#open text file
try:
    os.mkdir(month)
except Exception:
    pass
text_file = open(os.path.join(month, "data_" + indicatorCodeAI + ".json"), "w")
#write string to file
n = text_file.write(finalJson)

#close file
text_file.close()
print(" creado respaldo: " + text_file.name)
newIdsDict = {"newIds": newIds}
newIdsDf = pd.DataFrame(newIdsDict)
newIdsDf.to_csv(os.path.join(month, "new_ids_" + indicatorCodeAI + ".csv"))

 creado respaldo: JUNIO\data_WA1_01.json


In [15]:
## envio a AI
if (test):
    print(
        '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
else:
    try:
        if len(changesList) > 0:
            print('se envia a AI')
            client = Client(token=osmosys.osmosys.getToken(), base_url='https://www.activityinfo.org/resources')
            client.post_resource(path='update', body=finalJson)
            print(
                '--------------------------------------' + indicatorCodeAI + '------------------------------------------------------')
        else:
            print('nada que enviar a AI')
    except HTTPError as e:
        code = e.response.status_code
        print('error:')
        print(code)

        print(e)

--------------------------------------WA1_01------------------------------------------------------
